<a href="https://colab.research.google.com/github/Brackly/GNN-for-fraud-detection-/blob/main/GNN_for_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/content/fraudTrain.csv',on_bad_lines='skip')

In [ ]:
data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495.0,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149.0,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154.0,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1.325376e+09,43.150704,-112.154481,0.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939.0,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1.325376e+09,47.034331,-112.561071,0.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99.0,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1.325376e+09,38.674999,-78.632459,0.0


In [ ]:
import datetime

def time_to_seconds(time_str):
    time_obj = datetime.datetime.strptime(time_str, '%H:%M:%S').time()
    return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second

def seconds_since_midnight(datetime_str):
    date_str, time_str = datetime_str.split(' ')
    seconds = time_to_seconds(time_str)
    return seconds

# Apply the seconds_since_midnight function to the datetime column
data['seconds_since_midnight'] = data['trans_date_trans_time'].apply(seconds_since_midnight)

In [ ]:
data.dropna(subset=['is_fraud'], inplace=True)
from imblearn.over_sampling import RandomOverSampler

# Split the dataset into features (X) and target (y)
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']

# Create a RandomOverSampler object and fit it to the data
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Create a new DataFrame with the resampled data
df_resampled = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)
print(df_resampled['is_fraud'].value_counts())

0.0    30841
1.0    30841
Name: is_fraud, dtype: int64


In [ ]:
data['category'].unique()

array(['misc_net', 'grocery_pos', 'entertainment', 'gas_transport',
       'misc_pos', 'grocery_net', 'shopping_net', 'shopping_pos',
       'food_dining', 'personal_care', 'health_fitness', 'travel',
       'kids_pets', 'home'], dtype=object)

In [ ]:
data.columns

Index(['Unnamed: 0', '-75.9703', '3066', 'Product manager', '1928-07-15',
       '6ab9aa374e6c6659b5b0e3d95a295a26', '1327922752', '39.148413',
       '-76.802509', '0'],
      dtype='object')

In [ ]:
df_resampled['merchant_id'], _ = pd.factorize(df_resampled['merchant'])

In [ ]:
df_resampled.drop(columns=['trans_date_trans_time','first', 'last', 'gender', 'street', 'city', 'state', 'zip','job', 'dob','unix_time',],inplace=True)

In [ ]:
df_resampled.head()

,Unnamed: 0,cc_num,merchant,category,amt,lat,long,city_pop,trans_num,merch_lat,merch_long,seconds_since_midnight,is_fraud,merchant_id
0,0,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,36.0788,-81.1781,3495.0,0b242abb623afc578575680df30655b9,36.011293,-82.048315,18,0.0,0
1,1,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,48.8878,-118.2105,149.0,1f76529f8574734946361c461b024d99,49.159047,-118.186462,44,0.0,1
2,2,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,42.1808,-112.2620,4154.0,a1a22d70485983eac12b5b88dad1cf95,43.150704,-112.154481,51,0.0,2
3,3,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,46.2306,-112.1138,1939.0,6b849c168bdad6f867558c3793159a81,47.034331,-112.561071,76,0.0,3
4,4,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,38.4207,-79.4629,99.0,a41d7549acf90789359a9aa5346dcb46,38.674999,-78.632459,186,0.0,4


In [ ]:
import torch

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA device assigned:', device)
else:
    device = 'cpu'
    print('CUDA is not available')

CUDA device assigned: cuda


In [ ]:
def one_hot_encode(df, col):
    encoded = pd.get_dummies(df[col], prefix=col)

    # Concatenate the one-hot encoding to the original DataFrame
    df = pd.concat([df, encoded], axis=1)

    df.drop(col, axis=1, inplace=True)

    return df
df_resampled = one_hot_encode(df_resampled, 'category')

##Graph

In [ ]:
from torch import Tensor
print(torch.__version__)
!pip install sentence-transformers
!pip install torch_geometric
import os
os.environ['TORCH'] = torch.__version__
!pip install torch-sparse
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

2.0.0+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=12927fe36b367b255d2d0a6664392d68182ce7e5c6e894fea3978699cc372083
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━

In [ ]:
df_resampled.columns

Index(['Unnamed: 0', 'cc_num', 'merchant', 'amt', 'lat', 'long', 'city_pop',
       'trans_num', 'merch_lat', 'merch_long', 'seconds_since_midnight',
       'is_fraud', 'merchant_id', 'category_entertainment',
       'category_food_dining', 'category_gas_transport',
       'category_grocery_net', 'category_grocery_pos',
       'category_health_fitness', 'category_home', 'category_kids_pets',
       'category_misc_net', 'category_misc_pos', 'category_personal_care',
       'category_shopping_net', 'category_shopping_pos', 'category_travel'],
      dtype='object')

In [ ]:

credit_cards_df = df_resampled[['cc_num', 'lat', 'long']] \
                    .drop_duplicates(subset=['cc_num'])
merchants_df = df_resampled[['merchant_id', 'merch_lat', 'merch_long']] \
                    .drop_duplicates(subset=['merchant_id'])

In [ ]:
merchants_df.head()

,merchant_id,merch_lat,merch_long
0,0,36.011293,-82.048315
1,1,49.159047,-118.186462
2,2,43.150704,-112.154481
3,3,47.034331,-112.561071
4,4,38.674999,-78.632459


In [ ]:
transactions_path='transactions.csv'
credit_cards_path='credit_cards.csv'
merchants_path='merchants_cards.csv'
df_resampled.to_csv(transactions_path)
credit_cards_df.to_csv(credit_cards_path, index=False)
merchants_df.to_csv(merchants_path, index=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
class ScalerEncoder(object):
    def __init__(self, feature_range=(0, 1)):
        self.feature_range = feature_range
        self.scaler = MinMaxScaler(feature_range=feature_range)

    def __call__(self, df):
        x = self.scaler.fit_transform(df.values.reshape(-1, 1))
        return torch.tensor(x, dtype=torch.float32)

def load_node_csv(path, index_col, encoders=None,**kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    encoded= None
    if encoders is not None:
      xs = [encoder(df[col]) for col, encoder in encoders.items()]
      x = torch.cat(xs, dim=-1)
    return x, mapping

In [ ]:
credit_card, credit_card_mapping = load_node_csv(
    credit_cards_path, index_col='cc_num',
    # encoders={
        # 'lat': ScalerEncoder(),
        # 'long': ScalerEncoder(),
    # }
    )

In [ ]:
merchant, merchant_mapping = load_node_csv(
    merchants_path, index_col='merchant_id',
    # encoders={
        # 'merch_lat': ScalerEncoder(),
        # 'merch_long': ScalerEncoder(),
    # }
    )

In [ ]:
from torch_geometric.data import HeteroData

data = HeteroData()

In [ ]:
data['credit_card'].x = credit_card
data['merchant'].x = merchant
data["credit_card"].node_id = torch.arange(len(credit_card))
data["merchant"].node_id = torch.arange(len(merchant))

TypeError: ignored

In [ ]:
data

In [ ]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

In [ ]:
class IdentityEncoder(object):
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

import torch_geometric.transforms as T

edge_index, edge_label = load_edge_csv(
    transactions_path,
    src_index_col='cc_num',
    src_mapping=credit_card_mapping,
    dst_index_col='merchant_id',
    dst_mapping=merchant_mapping,
    encoders={'is_fraud': IdentityEncoder(dtype=torch.long)},
)

In [ ]:
import torch_geometric.transforms as T
data['credit_card', 'is_used_at', 'merchant'].edge_index = edge_index
data['credit_card', 'is_used_at', 'merchant'].edge_label = edge_label
data = T.ToUndirected()(data)

In [ ]:
data

HeteroData(
  credit_card={
    x=[909, 2],
    node_id=[909]
  },
  merchant={
    x=[693, 2],
    node_id=[693]
  },
  (credit_card, is_used_at, merchant)={
    edge_index=[2, 61682],
    edge_label=[61682, 1]
  },
  (merchant, rev_is_used_at, credit_card)={
    edge_index=[2, 61682],
    edge_label=[61682, 1]
  }
)

In [ ]:
from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
edge_label_index = data["credit_card", "is_used_at", "merchant"].edge_index
edge_label = data["credit_card", "is_used_at", "merchant"].edge_label

train_loader = LinkNeighborLoader(
    data=data,  # TODO
    num_neighbors=[20,10],  # TODO
    neg_sampling_ratio=2,  # TODO
    edge_label_index=(("credit_card", "is_used_at", "merchant"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  credit_card={
    x=[903, 2],
    node_id=[903],
    n_id=[903]
  },
  merchant={
    x=[693, 2],
    node_id=[693],
    n_id=[693]
  },
  (credit_card, is_used_at, merchant)={
    edge_index=[2, 9744],
    edge_label=[384, 1],
    e_id=[9744],
    input_id=[128],
    edge_label_index=[2, 384]
  },
  (merchant, rev_is_used_at, credit_card)={
    edge_index=[2, 11062],
    edge_label=[11062, 1],
    e_id=[11062]
  }
)


/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:58: UserWarning: Using '{self.__class__.__name__}' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn("Using '{self.__class__.__name__}' without a "


In [ ]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x
# The final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_credit_card: Tensor, x_merchant: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_credit_card = x_credit_card[edge_label_index[0]]
        edge_feat_merchant = x_merchant[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_credit_card * edge_feat_merchant).sum(dim=-1)


In [ ]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for customers and products:
        self.merchant_lin = torch.nn.Linear(data["merchant"].x.size(1), hidden_channels)
        self.credit_card_lin = torch.nn.Linear(data["credit_card"].x.size(1), hidden_channels)
        self.credit_card_emb = torch.nn.Embedding(data["credit_card"].num_nodes, hidden_channels)
        self.merchant_emb = torch.nn.Embedding(data["merchant"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "credit_card": self.credit_card_lin(data["credit_card"].x) + self.credit_card_emb(data["credit_card"].node_id),
          "merchant": self.merchant_lin(data["merchant"].x) + self.merchant_emb(data["merchant"].node_id),
        }

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)

        pred = self.classifier(
            x_dict["credit_card"],
            x_dict["merchant"],
            data["credit_card", "is_used_at", "merchant"].edge_label_index,
        )

        return pred


model = Model(hidden_channels=32)

print(model)

Model(
  (merchant_lin): Linear(in_features=2, out_features=32, bias=True)
  (credit_card_lin): Linear(in_features=2, out_features=32, bias=True)
  (credit_card_emb): Embedding(909, 32)
  (merchant_emb): Embedding(693, 32)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (credit_card__is_used_at__merchant): SAGEConv(32, 32, aggr=mean)
      (merchant__rev_is_used_at__credit_card): SAGEConv(32, 32, aggr=mean)
    )
    (conv2): ModuleDict(
      (credit_card__is_used_at__merchant): SAGEConv(32, 32, aggr=mean)
      (merchant__rev_is_used_at__credit_card): SAGEConv(32, 32, aggr=mean)
    )
  )
  (classifier): Classifier()
)


In [ ]:
import tqdm
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 7):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()

        # TODO: Move `sampled_data` to the respective `device`
        sampled_data = sampled_data.to(device)
        # TODO: Run `forward` pass of the model
        pred=model.forward(sampled_data)
        # TODO: Apply binary cross entropy via
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`
        loss = F.binary_cross_entropy_with_logits(pred, sampled_data["credit_card", "is_used_at", "merchant"]['edge_label'].squeeze().float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cuda'


100%|██████████| 482/482 [00:18<00:00, 25.43it/s]


Epoch: 001, Loss: -10255300.4832


100%|██████████| 482/482 [00:20<00:00, 23.17it/s]


Epoch: 002, Loss: -2721737867.8326


...

Epoch: 004, Loss: -294546126483.6351


100%|██████████| 482/482 [00:20<00:00, 23.79it/s]


Epoch: 005, Loss: -1109322544979.9443


100%|██████████| 482/482 [00:18<00:00, 25.48it/s]

Epoch: 006, Loss: -3076984924932.0923
